In [ ]:
# default_exp solution

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nangs.pde import PDE

# solutions

> This module contains the different function approximators available to solve PDEs.

In [ ]:
#export

import torch
import torch.nn as nn

def getActivation(a):
    if a == 'relu': return nn.ReLU(inplace=True)
    elif a == 'sigmoid': return nn.Sigmoid(inplace=True)
    else: raise Exception(f'activation function {a} not valid')

def block(i, o, a):
    return nn.Sequential(
        nn.Linear(i, o), 
        #nn.BatchNorm1d(o),
        getActivation(a)
    )

class Solution(nn.Module):
    "Currently, only MLPs are supported as solution approximators wiht same number of neurons and activation \
    function per layer"
    def __init__(self, inputs, outputs, layers, neurons, activations):
        super().__init__()
        
        # checks
        if not isinstance(inputs, int) or inputs <= 0: raise Exception('inputs must be a postive integer')
        if not isinstance(outputs, int) or outputs <= 0: raise Exception('outputs must be a positive integer')
        if not isinstance(layers, int) or layers <= 0: raise Exception('layers must be a positive integer')
        if not isinstance(neurons, int) or neurons <= 0: raise Exception('neurons must be a positive integer')
        if not isinstance(activations, str): raise Exception('activation must be a string')

        # activaton function
        self.activation = activations
        # layers
        self.fc_in = block(inputs, neurons, self.activation)
        self.fc_hidden = nn.ModuleList()
        for layer in range(layers):
            self.fc_hidden.append(block(neurons, neurons, self.activation))
        self.fc_out = nn.Linear(neurons, outputs)
    

    def forward(self, x):
        x = self.fc_in(x)
        for layer in self.fc_hidden:
            x = layer(x)
        x = self.fc_out(x)
        return x

Example

In [ ]:
pde = PDE(inputs=['x', 't'], outputs=['p'])

# add values and bocos

mlp = {'layers': 3, 'neurons': 100, 'activations': 'relu'}
pde.buildModel(mlp)

2


In [ ]:
#hide

try:
    mlp = {'layers': 3.2, 'neurons': 100, 'activations': 'relu'}
    pde.buildModel(mlp)
except Exception as e:
    assert str(e) == "layers must be a positive integer", "assertion failed"
    
try:
    mlp = {'layers': 3, 'neurons': 100.2, 'activations': 'relu'}
    pde.buildModel(mlp)
except Exception as e:
    assert str(e) == "neurons must be a positive integer", "assertion failed"
    
try:
    mlp = {'layers': -3, 'neurons': 100, 'activations': 'relu'}
    pde.buildModel(mlp)
except Exception as e:
    assert str(e) == "layers must be a positive integer", "assertion failed"
    
    
try:
    mlp = {'layers': 3, 'neurons': 100, 'activations': 'abc'}
    pde.buildModel(mlp)
except Exception as e:
    assert str(e) == "activation function abc not valid", "assertion failed"

2
2
2
2
